In [1]:
from load_yelp_data import load_yelp_dataframe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
businesses = load_yelp_dataframe('businesses')
reviews = load_yelp_dataframe('reviews')
users = load_yelp_dataframe('users')

In [9]:
users.head(2)

,Unnamed: 0,average_stars,compliments,elite,fans,friends,name,review_count,type,user_id,votes,yelping_since
0,0,4.14,"{'plain': 25, 'writer': 9, 'cute': 15, 'photos...","[2005, 2006]",69,"[1, 2, 3, 5, 93, 12, 99, 464, 1025, 1298, 1388...",Russel,108,user,0,"{'cool': 246, 'useful': 282, 'funny': 167}",2004-10
1,1,3.67,"{'plain': 970, 'writer': 346, 'cute': 204, 'ph...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",1345,"[0, 2, 3, 4, 5, 6, 8, 9, 12, 95, 97, 187, 465,...",Jeremy,1292,user,1,"{'cool': 12091, 'useful': 15242, 'funny': 8399}",2004-10


In [10]:
reviews.head(2)

,Unnamed: 0,user_id,business_id,year,month,day,stars
0,0,511457,0,2012,8,1,4
1,1,291005,0,2014,2,13,5


In [11]:
businesses.head(2)

,Unnamed: 0,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,0,"{'Accepts Credit Cards': True, 'Noise Level': ...",0,"['Fast Food', 'Restaurants']",Dravosburg,"4734 Lebanon Church Rd\nDravosburg, PA 15034","{'Monday': {'close': '21:00', 'open': '11:00'}...",40.354327,-79.900706,Mr Hoagie,[],True,7,3.5,PA,business
1,1,"{'Accepts Credit Cards': True, 'Price Range': ...",1,['Nightlife'],Dravosburg,"202 McClure St\nDravosburg, PA 15034",{},40.350553,-79.886814,Clancy's Pub,[],True,5,3.0,PA,business
